# Utilizando el servidor de nombres

El servidor de nombres permite que un cliente se contacte con un serivor de manera más sencilla.

Cuándo no se utiliza un servidor de nombres, se necesita especificar la IP y puerto del objeto remoto
en cada llamada.

In [1]:
from IPython.display import IFrame
IFrame(
    'https://pythonhosted.org/Pyro4/servercode.html#intermission-example-2-server-and-client-with-name-server',
    width='100%', height='500px')

# Algunas consideraciones
Esta documento es un Notebook de Jupyter, cuyo kernel es un solo proceso.

Para poder lanzar procesos utilizaremos:

* Clase [`subporcess.Popen(comando)`](https://pythonhosted.org/Pyro4/servercode.html#intermission-example-2-server-and-client-with-name-server) para lanzar comandos en segundo plano. Lo utilizaremos para el comando `pyro4-ns`.
* Clase [`multiprocess.Process(target, args, kwargs)`](https://docs.python.org/2/library/multiprocessing.html) Permite lanzar un nuevo intérprete a a partir del estado actual, comenzando a ejecutar en la función `target`.
* Función [`atexit.register(funcion, *args, **kwargs)`](https://docs.python.org/2/library/atexit.html) Pemrite registrar funciones al momento de que un proceso termina. Lo utilizaremos para detener el servidor y el servidor de nombres al momento de que el kernel se cierra.
* Métodos `start()` y `stop()` en `Process`.
* Ejecución de líneas de bash mediante el comando `!` de Jupyter. Las variables de bash se referencias con `$nombre`.
* Comando [`lsof`](http://linux.die.net/man/8/lsof) para detectar que procesos están escuchando en un puerto. En particular, el puerto del servidor de nombres.
* Comando `kill -KILL` para terminar un proceso.

In [2]:
# Mater posibles nameservers
pids = !lsof -t -i TCP:9090 -sTCP:LISTEN
for pid in pids:
    print("Matando a %s" % pid)
    !kill -KILL $pid && sleep 2
    
# Lanzar el servidor de nomrbes en segundo plano
import atexit
from subprocess import Popen
nameserver = Popen('pyro4-ns')
def al_finalizar(servidor):
    print("Finalizando el servidor de nombres con pid %s", servidor.pid)
    nameserver.terminate()
atexit.register(al_finalizar, servidor=nameserver) # Matamos al NS al final
print("Servidor de nombres en ejecución %d" % nameserver.pid)

Servidor de nombres en ejecución 48950


### *El código de arriba es el equivalente a lanzarlo desde la consola:*
![Servidor de Nombres de Pyro4](static/pyro-ns.png)

Si lo lanzamos desde la consola, el servidor de nombres deberá estar en ejecución duranto todo el proceo de desarollo y eventual puesta en producción. No se activa como servicio del sistema por defecto, a diferencia de  [`rpcbind`](https://en.wikipedia.org/wiki/Portmap).

# Creando el servidor

In [3]:
import os
from Pyro4 import Daemon
from socket import gethostname

hostname = gethostname()

class Saludador(object):
    """
    Objeto a ser expuesto.
    """
    def saludar(self, nombre):
        """
        Retorna un saludo
        """
        return "Hola %s" % nombre
    
def exponer_objetos():
    # Crear una instancia
    saludador = Saludador()
    # serverSimple activa el loop, en este caso el objeto
    # se expone bajo el nombre saludador.
    Daemon.serveSimple(
    {
        saludador: "saludador"
    },
    ns=True, verbose=True, host=hostname)


### Creación de un proceso servidor

In [4]:
# Utilizamos la función definida en la celda anterior
from multiprocessing import Process
servidor = Process(target=exponer_objetos)
servidor.start()
atexit.register(servidor.terminate)
print("El servidor es un subproceso con el PID: %d" % servidor.pid)

El servidor es un subproceso con el PID: 48962


# Ahora el cliente

In [5]:
from Pyro4 import Proxy

Object <__main__.Saludador object at 0x1089adb10>:
    uri = PYRO:obj_1e4822f1b3654f2abc1acf185c4f29f1@greybird:55915
    name = saludador
Pyro daemon running.


## Ahora el serivdor de nombres se encarga de mantener las ubicaciones

In [6]:
objeto = Proxy('PYRONAME:saludador')

In [7]:
objeto.saludar('Pepe')

u'Hola Pepe'

![Success](static/meme_success.jpg)